<a href="https://colab.research.google.com/github/josemanuelvinhas/MarvelRecomverse/blob/main/MarvelRecomverse_API_de_Marvel_con_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. API de Marvel Comics**

La API de Marvel Comics es una herramienta para ayudar a los desarrolladores de todo el mundo a crear sitios web y aplicaciones utilizando datos de los más de 70 años de la era de los cómics Marvel. Se trata de un servicio RESTful que proporciona métodos para acceder a recursos específicos meidante URL. Permite buscar y filtrar conjuntos de recursos según varios criterios. Los resultados se devuelven en formato JSON.

Para hacer uso de la API es necesario registrarse, y de esta manera obtener una API Key. Esto se puede hacer desde [Marvel Developer Portal](https://developer.marvel.com/).

Una vez que se obtiene la API Key se puede probar la API mediante la herramienta [Interactive API Tester](https://developer.marvel.com/docs).

# **2. MarvelRecomverse: Uso de la API con python**

La aplicación a desarrollar necesita un dataset de personajes de cómics de Marvel y sus descripciones. Para crear este dataset se hará uso de la API de Marvel usando python.

Para obtener la información necesaria las peticiones deberán tener las siguientes características:

- Tipo: GET
- URL: https://gateway.marvel.com/v1/public/characters
- Parámetros: 
  1. *limit* : es el número máximo de personajes que devuelve la consulta (máx. 100)
  2. *offset* : debido al límite de personajes que devuelve la consulta, es necesario indicar a partir de qúe personaje queremos que la consulta nos devuelva resultados. Por ejemplo, si existen 300 personajes y queremos obtener los que van del 200 al 300 debemos indicar un *limit=100* y un *offset=200*
  3. *ts* : sello de tiempo
  4. *apikey* : la clave pública que proporciona Marvel
  5. *hash* : un hash de *Sello de Tiempo + Clave Privada + Clave Pública*

**1. Librerías:** se emplearán 4 librerías
- *request*: para realizar peticiones 
- *datetime*: para obtener el Sello de Tiempo
- *hashlib*: para realizar el hash requerido
- *pandas*: para manejar el dataset


In [1]:
import requests
import datetime
import hashlib
import pandas

**2. Construyendo la petición**
- *Sello de Tiempo*: se obtiene al comienzo del script mediante la librería *datetime*
- *API Key*: se proporcionan 2 claves, una publica, que se enviará como parámetro, y otra privada, que se emplea en el Hash.
- *Hash*: se obtendrá el *hash* usando el algoritmo md5 y obteniendo el resumen en hexadecimal 

In [2]:
timestamp = datetime.datetime.now().strftime('%Y-%m-%d%H:%M:%S')
public_api_key = 'd5d4682141673489b856445431f75246'
private_api_key = '8687323dce2867e5aab980b8dde10cb7e6f3d360'

hash_md5 = hashlib.md5()
hash_md5.update(f'{timestamp}{private_api_key}{public_api_key}'.encode('utf-8'))
hash = hash_md5.hexdigest()

- *limit*: como el objetivo es obtener todos los personajes este parámetro será el máximo posible (100)
- *offset*: como el objetivo es obtener todos los personajes este parámetro comenzará siendo 0 e irá avanzando de 100 en 100 hasta llegar a 1500, ya que hay un poco más de 1400 personajes.

**3. Realizando as consultas**

Se desea guardar el nombre y la descripción de cada personaje, para el cual se usarán 2 diccionarios. Se recorrerán los resultados obtenidos en cada petición y los personajes cuya descripción no sea vacía se guardarán.

Se empleará la librería *requests* para realizar la petición get con los parámetros deseados.

In [3]:
name = []
description = []

for i in range(0, 1500, 100):
    parametros = {'limit': 100, 'offset': i, 'ts': timestamp, 'apikey': public_api_key, 'hash': hash}
    print("Consulta a realizar con los siguientes parametros: ",parametros)
    respuesta = requests.get('https://gateway.marvel.com:443/v1/public/characters', parametros)
    resultados = respuesta.json()
    for personaje in resultados['data']['results']:
        if (personaje['description'].strip() != ""):
            name.append(personaje['name'])
            description.append(personaje['description'])

Consulta a realizar con los siguientes parametros:  {'limit': 100, 'offset': 0, 'ts': '2020-11-1914:30:13', 'apikey': 'd5d4682141673489b856445431f75246', 'hash': 'bdf8b4b0aadaa61e713722f16cb07e21'}
Consulta a realizar con los siguientes parametros:  {'limit': 100, 'offset': 100, 'ts': '2020-11-1914:30:13', 'apikey': 'd5d4682141673489b856445431f75246', 'hash': 'bdf8b4b0aadaa61e713722f16cb07e21'}
Consulta a realizar con los siguientes parametros:  {'limit': 100, 'offset': 200, 'ts': '2020-11-1914:30:13', 'apikey': 'd5d4682141673489b856445431f75246', 'hash': 'bdf8b4b0aadaa61e713722f16cb07e21'}
Consulta a realizar con los siguientes parametros:  {'limit': 100, 'offset': 300, 'ts': '2020-11-1914:30:13', 'apikey': 'd5d4682141673489b856445431f75246', 'hash': 'bdf8b4b0aadaa61e713722f16cb07e21'}
Consulta a realizar con los siguientes parametros:  {'limit': 100, 'offset': 400, 'ts': '2020-11-1914:30:13', 'apikey': 'd5d4682141673489b856445431f75246', 'hash': 'bdf8b4b0aadaa61e713722f16cb07e21'}
Co

**4. Almacenando los resultados**

Una vez se obtienen los resultados se guardarán en un documento *.csv* haciendo uso de la librería *pandas*.

*Nota: se guardarán los datos sin almacenar el índice, ya que se va a hacer uso del archivo usando pandas, que ya añade el índice*

In [4]:
data = {'name': name, 'description': description}
dataframe = pandas.DataFrame(data)
dataframe.to_csv("marvel.csv", index=False)

dataframe

,name,description
0,A-Bomb (HAS),Rick Jones has been Hulk's best bud since day ...
1,A.I.M.,AIM is a terrorist organization bent on destro...
2,Abomination (Emil Blonsky),"Formerly known as Emil Blonsky, a spy of Sovie..."
3,Adam Warlock,Adam Warlock is an artificially created human ...
4,Agent X (Nijo),Originally a partner of the mind-altering assa...
...,...,...
276,Zarek,Zarek is a member of the Kree race with no sup...
277,Zodiak,"Twelve demons merged with Norman Harrison, who..."
278,Zombie (Simon Garth),War hero Simon Garth was turned into a zombie ...
279,Zuras,Zuras was once the leader of the Eternals.


#**3. Añadir, Modificar y Borrar un personaje del dataset**

Para **añadir** un personaje al final del DataFrame

1.   Se cargan los datos del csv
2.   Se añade el nuevo personaje con el método *append()*
3.   Se guardan los datos

Añadir a *Mr. Yomiquesh*:


In [5]:
import pandas as pd

originalData = pd.read_csv('marvel.csv')
originalData = originalData.append({'name' : 'Mr. Yomiquesh', 'description' : 'Yomiquesh was once the leader of the Eternals. He is strong as iron and as small as an ant. He is a terrorist'}, ignore_index=True)
originalData.to_csv("marvel_add.csv", index=False)

originalData

,name,description
0,A-Bomb (HAS),Rick Jones has been Hulk's best bud since day ...
1,A.I.M.,AIM is a terrorist organization bent on destro...
2,Abomination (Emil Blonsky),"Formerly known as Emil Blonsky, a spy of Sovie..."
3,Adam Warlock,Adam Warlock is an artificially created human ...
4,Agent X (Nijo),Originally a partner of the mind-altering assa...
...,...,...
277,Zodiak,"Twelve demons merged with Norman Harrison, who..."
278,Zombie (Simon Garth),War hero Simon Garth was turned into a zombie ...
279,Zuras,Zuras was once the leader of the Eternals.
280,Zzzax,"A chain reaction in an atomic reactor, a resul..."


Para **modificar** un personaje al final del DataFrame

1.   Se cargan los datos del csv
3.   Se modifica el personaje
3.   Se guardan los datos

Modificar la descripción de *Mr. Yomiquesh*:

In [6]:
import pandas as pd

originalData = pd.read_csv('marvel_add.csv')
originalData.loc[originalData['name'] == 'Mr. Yomiquesh', 'description'] = 'Yomiquesh was once the leader of the Eternals.'
originalData.to_csv("marvel_mod.csv", index=False)

originalData

,name,description
0,A-Bomb (HAS),Rick Jones has been Hulk's best bud since day ...
1,A.I.M.,AIM is a terrorist organization bent on destro...
2,Abomination (Emil Blonsky),"Formerly known as Emil Blonsky, a spy of Sovie..."
3,Adam Warlock,Adam Warlock is an artificially created human ...
4,Agent X (Nijo),Originally a partner of the mind-altering assa...
...,...,...
277,Zodiak,"Twelve demons merged with Norman Harrison, who..."
278,Zombie (Simon Garth),War hero Simon Garth was turned into a zombie ...
279,Zuras,Zuras was once the leader of the Eternals.
280,Zzzax,"A chain reaction in an atomic reactor, a resul..."


Para **borrar** un personaje al final del DataFrame

1.   Se cargan los datos del csv
2.   Se obtiene el índice del personaje
3.   Se borra el personaje
3.   Se guardan los datos

Eliminar a *Mr. Yomiquesh*:

In [7]:
import pandas as pd

originalData = pd.read_csv('marvel_add.csv')
indexOfCharacter = originalData[originalData['name']=="Mr. Yomiquesh"].index.values[0]
originalData = originalData.drop(indexOfCharacter, axis=0)
originalData.to_csv("marvel_delete.csv", index=False)

originalData

,name,description
0,A-Bomb (HAS),Rick Jones has been Hulk's best bud since day ...
1,A.I.M.,AIM is a terrorist organization bent on destro...
2,Abomination (Emil Blonsky),"Formerly known as Emil Blonsky, a spy of Sovie..."
3,Adam Warlock,Adam Warlock is an artificially created human ...
4,Agent X (Nijo),Originally a partner of the mind-altering assa...
...,...,...
276,Zarek,Zarek is a member of the Kree race with no sup...
277,Zodiak,"Twelve demons merged with Norman Harrison, who..."
278,Zombie (Simon Garth),War hero Simon Garth was turned into a zombie ...
279,Zuras,Zuras was once the leader of the Eternals.
